In [ ]:
#ZOTTOLA GABRIELE 812363
#LEUCE FRANCESCO 794407

In [ ]:
import pandas as pd 
import numpy as np 
from datetime import datetime 
from pandas.tseries.offsets import YearBegin, Day  

In [ ]:
#1)Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
#carico il dataset 'loans_lenders' e osservo tramite il comando head che nella riga lenders sono presenti più lenders
#separati da virgola 
loans_lenders = pd.read_csv ('/content/drive/My Drive/loans_lenders.csv')
loans_lenders.head()

,loan_id,lenders
0,483693,"muc888, sam4326, camaran3922, lachheb1865, reb..."
1,483738,"muc888, nora3555, williammanashi, barbara5610,..."
2,485000,"muc888, terrystl, richardandsusan8352, sherri4..."
3,486087,"muc888, james5068, rudi5955, daniel9859, don92..."
4,534428,"muc888, niki3008, teresa9174, mike4896, david7..."


In [ ]:
#dividiamo i lenders per ogni loan id in modo tale da normalizzare il dataset utilizzando il comando sottostante
#utilizzo per separare la funzione str.split, specificando come sepratore tra le parentesi la virgola ','. 
#la funzione explode trasforma ogni elemento della lista in una riga copiando il rispettivo indice
loans_lenders = loans_lenders.assign(lenders = loans_lenders['lenders'].str.split(',')).explode('lenders')
loans_lenders.head()

,loan_id,lenders
0,483693,muc888
0,483693,sam4326
0,483693,camaran3922
0,483693,lachheb1865
0,483693,rebecca3499


In [ ]:
#2) For each loan, add a column duration
#corresponding to the number of days between 
#the disburse time and the planned expiration time. 
#If any of those two dates is missing, also the 
#duration must be missing.

#per prima cosa carico il dataset loans, di tale dataset isolo unicamente gli attributi che mi servono 
#nel calcolo (disburse_time e planned_expiration_time), dopodichè converto gli attributi in "datetime".
#Una volta calcolata la differenza richiesta isolo i valori Nan per osservare se anche la differenza dava il risultao richiesto (Nan)

In [ ]:
#carico il dataframe loans e tramite funzione info() osservo gli attributi presenti 
loans = pd.read_csv ('/content/drive/My Drive/loans.csv')
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1419607 entries, 0 to 1419606
Data columns (total 31 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   loan_id                          1419607 non-null  int64  
 1   loan_name                        1372049 non-null  object 
 2   original_language                1376356 non-null  object 
 3   description                      1376322 non-null  object 
 4   description_translated           966929 non-null   object 
 5   funded_amount                    1419607 non-null  float64
 6   loan_amount                      1419607 non-null  float64
 7   status                           1419607 non-null  object 
 8   activity_name                    1419607 non-null  object 
 9   sector_name                      1419607 non-null  object 
 10  loan_use                         1376334 non-null  object 
 11  country_code                     1419598 non-null 

In [ ]:
#per maggiore chiarezza isolo le colonne per il calcolo della differenza 
loans[['disburse_time', 'planned_expiration_time']]

,disburse_time,planned_expiration_time
0,2013-12-22 08:00:00.000 +0000,2014-02-14 03:30:06.000 +0000
1,2013-12-20 08:00:00.000 +0000,2014-03-26 22:25:07.000 +0000
2,2014-01-09 08:00:00.000 +0000,2014-02-15 21:10:05.000 +0000
3,2014-01-17 08:00:00.000 +0000,2014-02-21 03:10:02.000 +0000
4,2013-12-17 08:00:00.000 +0000,2014-02-13 06:10:02.000 +0000
...,...,...
1419602,2015-11-23 08:00:00.000 +0000,2016-01-02 01:00:03.000 +0000
1419603,2015-11-24 08:00:00.000 +0000,2016-01-02 16:40:07.000 +0000
1419604,2015-11-13 08:00:00.000 +0000,2016-01-03 22:20:04.000 +0000
1419605,2015-11-03 08:00:00.000 +0000,2016-01-05 08:50:02.000 +0000


In [ ]:
#osservo il tipo di oggetto con cui andiamo a lavorare
#per poter fare una differenza tra date l'oggetto deve essere di tipo datetime
#converto quindi gli attributi in versione datetime
type(loans['disburse_time'])
type(loans['planned_expiration_time'])
loans['disburse_time_dateversion'] = pd.to_datetime(loans['disburse_time'], format= "%Y-%m-%d %H:%M:%S.%f", errors='raise')
loans['planned_expiration_time_dateversion'] = pd.to_datetime(loans['planned_expiration_time'], format= "%Y-%m-%d %H:%M:%S.%f", errors='raise')

In [ ]:
#calcolo della differenza richiesta tra 'planned_expiration_time_dateversion' e 'disburse_time_dateversion' 
loans['duration'] = loans['planned_expiration_time_dateversion'] - loans['disburse_time_dateversion']
loans.head()

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_dateversion,planned_expiration_time_dateversion,duration
0,657307,Aivy,English,"Aivy, 21 years of age, is single and lives in ...",NaN,125.0,125.0,funded,General Store,Retail,to buy additional inventories for her sari-sa...,PH,Philippines,Ozamiz - Ozamiz City,shared,0.1,PHP,126.0,2014-01-15 02:23:45.000 +0000,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,2014-01-15 04:48:22.000 +0000,7.0,3,2,1,NaN,female,true,irregular,field_partner,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,53 days 19:30:06
1,657259,Idalia Marizza,Spanish,"Doña Idalia, esta casada, tiene 57 años de eda...","Idalia, 57, is married and lives with her husb...",400.0,400.0,funded,Used Clothing,Clothing,"To buy American clothing such as blouses, shir...",HN,Honduras,"La Lopez, Choloma",shared,0.1,HNL,201.0,2014-01-14 20:23:20.000 +0000,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,2014-02-25 06:42:06.000 +0000,8.0,11,2,1,NaN,female,true,monthly,field_partner,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,96 days 14:25:07
2,658010,Aasia,English,Aasia is a 45-year-old married lady and she ha...,NaN,400.0,400.0,funded,General Store,Retail,"to buy stock of rice, sugar and flour",PK,Pakistan,Lala Musa,shared,0.1,PKR,245.0,2014-01-16 11:32:58.000 +0000,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,2014-01-24 23:06:18.000 +0000,14.0,16,2,1,"#Woman Owned Biz, #Supporting Family, user_fav...",female,true,monthly,field_partner,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,37 days 13:10:05
3,659347,Gulmira,Russian,"Гулмире 36 лет, замужем, вместе с супругом вос...",Gulmira is 36 years old and married. She and ...,625.0,625.0,funded,Farming,Agriculture,"to buy cucumber and tomato seeds, as well as f...",KG,Kyrgyzstan,"Aravan village, Osh region",shared,0.1,KGS,171.0,2014-01-20 09:59:48.000 +0000,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,2014-01-22 05:29:28.000 +0000,14.0,21,2,1,user_favorite,female,true,monthly,field_partner,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,34 days 19:10:02
4,656933,Ricky\t,English,Ricky is a farmer who currently cultivates his...,NaN,425.0,425.0,funded,Farming,Agriculture,to buy organic fertilizer and agrochemical pr...,PH,Philippines,"Baleleng, Sto. Thomas, Isabela",shared,0.1,PHP,123.0,2014-01-14 05:46:21.000 +0000,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,2014-01-14 17:29:27.000 +0000,7.0,15,2,1,"#Animals, #Eco-friendly, #Sustainable Ag",male,true,bullet,field_partner,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,57 days 22:10:02


In [ ]:
#osserviamo se la differenza ha valore null in presenza di valori nulli "disburse_time" oppure "planned_expiration_time"
#per fare questo isolo le righe dove "disburse_time" e "planned_expiration_time" hanno valore null 
loans[loans['disburse_time_dateversion'].isnull()]
#si conferma che l'attributo duration ha valore null 

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_dateversion,planned_expiration_time_dateversion,duration
31106,1379508,Melissa,English,I grew up in Chicago and started playing viola...,I grew up in Chicago and started playing viola...,125.0,1000.0,expired,Education provider,Education,write the software that allows music teachers ...,US,United States,NaN,not shared,NaN,USD,NaN,2017-09-18 21:17:47.000 +0000,2017-12-06 16:41:36.000 +0000,NaN,NaN,12.0,2,1,1,NaN,female,true,bullet,direct,NaT,2017-12-06 16:41:36+00:00,NaT
31107,1428617,Dwarika,English,It all started when I was not conscious about ...,It all started when I was not conscious about ...,0.0,10000.0,fundRaising,Technology,Services,boost business advertisement and support work...,US,United States,NaN,not shared,NaN,USD,NaN,2017-12-05 18:27:03.000 +0000,2018-01-25 19:25:55.000 +0000,NaN,NaN,36.0,0,1,1,NaN,male,true,bullet,direct,NaT,2018-01-25 19:25:55+00:00,NaT
31178,1135319,Robert,English,"My name is Roberto Hernandez, I grew up in a s...","My name is Roberto Hernandez, I grew up in a s...",50.0,500.0,expired,Transportation,Transportation,purchase my own vehicle,US,United States,NaN,not shared,NaN,USD,NaN,2016-06-24 04:50:30.000 +0000,2016-09-02 20:27:46.000 +0000,NaN,NaN,24.0,2,1,1,NaN,male,true,bullet,direct,NaT,2016-09-02 20:27:46+00:00,NaT
32151,1279252,Michal,English,My name is Michal Szymczak and I am a Polish i...,My name is Michal Szymczak and I am a Polish i...,0.0,1000.0,expired,Services,Services,with establishing myself and being able to cre...,US,United States,NaN,not shared,NaN,USD,NaN,2017-04-14 12:34:14.000 +0000,2017-05-04 23:31:00.000 +0000,NaN,NaN,12.0,0,1,1,NaN,male,true,bullet,direct,NaT,2017-05-04 23:31:00+00:00,NaT
32152,1433349,NaN,NaN,NaN,NaN,0.0,8000.0,refunded,Services,Services,NaN,US,United States,NaN,not shared,NaN,USD,NaN,2017-12-13 11:24:56.000 +0000,2018-01-04 18:49:43.000 +0000,NaN,NaN,36.0,0,1,1,NaN,NaN,NaN,bullet,direct,NaT,2018-01-04 18:49:43+00:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419522,1082442,Joanna,English,"I am the youngest of four daughters, all in th...","I am the youngest of four daughters, all in th...",510.0,3000.0,expired,General Store,Retail,purchase stock to fulfill orders from new cust...,US,United States,NaN,not shared,NaN,USD,NaN,2014-08-05 19:50:28.000 +0000,2014-10-14 16:05:01.000 +0000,NaN,NaN,24.0,37,1,1,NaN,female,true,bullet,direct,NaT,2014-10-14 16:05:01+00:00,NaT
1419524,1227124,Donna & Brighton,English,"In 1979, I graduated from the University of Wy...","In 1979, I graduated from the University of Wy...",800.0,10000.0,expired,Retail,Retail,"our community become stronger , healthier and...",US,United States,NaN,not shared,NaN,USD,NaN,2017-01-21 03:58:42.000 +0000,2017-02-09 23:07:54.000 +0000,NaN,NaN,36.0,9,1,1,NaN,female,true,bullet,direct,NaT,2017-02-09 23:07:54+00:00,NaT
1419525,1089430,NaN,NaN,NaN,NaN,125.0,225.0,refunded,Grocery Store,Food,NaN,KE,Kenya,NaN,not shared,NaN,KES,NaN,2015-07-01 10:10:48.000 +0000,2015-09-21 08:06:19.000 +0000,NaN,NaN,6.0,3,1,1,NaN,NaN,NaN,bullet,direct,NaT,2015-09-21 08:06:19+00:00,NaT
1419538,1235452,Tara,English,Currently homeless.\r\nComplete alchemist.\r\n...,Currently homeless.\r\nComplete alchemist.\r\n...,600.0,3000.0,expired,Technology,Services,me to set the foundation and get my business g...,US,United States,NaN,not shared,NaN,USD,NaN,2017-02-09 22:58:24.000 +0000,2017-02-25 20:21:21.000 +0000,NaN,NaN,24.0,3,1,1,NaN,female,true,bullet,direct,NaT,2017-

In [ ]:
#osserviamo le righe dove "planned_expiration_time_dateversion" è un valore null
loans[loans['planned_expiration_time_dateversion'].isnull()]
#si conferma che l'attributo duration ha valore null

,loan_id,loan_name,original_language,description,description_translated,funded_amount,loan_amount,status,activity_name,sector_name,loan_use,country_code,country_name,town_name,currency_policy,currency_exchange_coverage_rate,currency,partner_id,posted_time,planned_expiration_time,disburse_time,raised_time,lender_term,num_lenders_total,num_journal_entries,num_bulk_entries,tags,borrower_genders,borrower_pictured,repayment_interval,distribution_model,disburse_time_dateversion,planned_expiration_time_dateversion,duration
64,251406,José Roan Group,Spanish,El grupo se llama “José Roan” y se ubica en l...,The group called “José Roan” is located in the...,825.0,825.0,funded,Poultry,Agriculture,Purchase young chickens and feed so that the a...,MX,Mexico,"COPAINALÁ, CHIAPAS",shared,0.2,MXN,130.0,2010-11-23 18:28:13.000 +0000,NaN,2010-11-18 08:00:00.000 +0000,2010-12-30 02:18:42.000 +0000,7.0,22,2,1,NaN,"female, female, female","true, true, true",monthly,field_partner,2010-11-18 08:00:00+00:00,NaT,NaT
65,252537,Marylou,English,"Marylou S. is from the village of Kuguita, Mam...",NaN,175.0,175.0,funded,Personal Products Sales,Retail,to purchase additional inventory and as workin...,PH,Philippines,"Kuguita, Mambajao, Camiguin",not shared,NaN,PHP,128.0,2010-11-26 07:28:51.000 +0000,NaN,2010-11-25 08:00:00.000 +0000,2010-11-27 18:22:17.000 +0000,7.0,7,2,1,NaN,female,true,irregular,field_partner,2010-11-25 08:00:00+00:00,NaT,NaT
66,253044,Flora,English,"Flora G. is from the village of Libona, Bukidn...",NaN,125.0,125.0,funded,Transportation,Transportation,To purchase tools needed to expand services pr...,PH,Philippines,"Libona, Bukidnon",not shared,NaN,PHP,128.0,2010-11-27 11:29:43.000 +0000,NaN,2010-11-25 08:00:00.000 +0000,2010-11-27 21:36:44.000 +0000,7.0,5,2,1,NaN,female,true,irregular,field_partner,2010-11-25 08:00:00+00:00,NaT,NaT
67,254012,Maria Magdalena,Spanish,"María de 55 años, tiene una tienda de víveres ...","María, 55, has a grocery store in the front pa...",1000.0,1000.0,funded,General Store,Retail,"For rice, sugar, oil.",EC,Ecuador,Guayaquil (Fortin),not shared,NaN,USD,147.0,2010-11-30 22:38:46.000 +0000,NaN,2010-11-16 08:00:00.000 +0000,2010-12-24 21:33:46.000 +0000,12.0,36,1,1,NaN,female,true,monthly,field_partner,2010-11-16 08:00:00+00:00,NaT,NaT
69,251336,Rober,Spanish,"Rober, de 32 años, está casado con Janeth, de ...","Rober, 32, is married to Janeth, 31, and they ...",375.0,375.0,funded,Knitting,Arts,To buy 15 bags of spools of thread.,PE,Peru,TARMA,shared,0.2,PEN,71.0,2010-11-23 15:22:39.000 +0000,NaN,2010-11-10 08:00:00.000 +0000,2010-11-24 07:18:30.000 +0000,12.0,15,3,2,NaN,male,true,monthly,field_partner,2010-11-10 08:00:00+00:00,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1418885,192543,"S.D.A. Primary School, Masanga Group",English,These are teachers of the SDA Primary School a...,NaN,1700.0,1700.0,funded,Personal Expenses,Personal Use,"To buy furniture like chairs, a bed, tables, a...",SL,Sierra Leone,"Masanga Village, Magburaka",shared,0.2,SLL,57.0,2010-04-12 17:47:02.000 +0000,NaN,2010-04-09 07:00:00.000 +0000,2010-04-20 14:57:27.000 +0000,11.0,56,1,1,NaN,"male, male, female, male, male, male, female","true, true, true, true, true, true, true",monthly,field_partner,2010-04-09 07:00:00+00:00,NaT,NaT
1418886,192953,Elpidio,English,"Elpidio B., from the village of barangay La To...",NaN,375.0,375.0,funded,Farming,Agriculture,To purchase fertilizers such as manure,PH,Philippines,"Brgy.La Torre, Bayombong,Nueva Vizcaya",shared,0.2,PHP,123.0,2010-04-14 06:38:48.000 +0000,NaN,2010-03-26 07:00:00.000 +0000,2010-05-09 23:17:23.000 +0000,8.0,14,2,1,NaN,male,true,bullet,field_partner,2010-03-26 07:00:00+00:00,NaT,NaT
1418887,193671,Deborah,English,Deborah is a married with seven children and ...,NaN,500.0,500.0,funded,Fish Selling,Food,To buy more baskets of fish,UG,Uganda,Kampala,shared,0.2,UGX,112.0,2010-04-16 10:11:46.

In [ ]:
#3)Find the lenders that have funded at least twice.
#per prima cosa unisco i due dataset (loans_lenders e loans) in modo tale da avere le informazioni in un unico dataframe. 
#Isolo tutte le istanze che hanno come satus 'funded' e verifico per ogni lenders (utilizzando la funzione groupby) la dimensione (size)
#dei loan assoiciati 
#con attributo di selezione >=2 osservo quali sono i lenders che hanno finanziato almeno due volte 
dataframe_unito = pd.merge(loans_lenders, loans[['loan_id', 'status']], on = 'loan_id')
dataframe_unito

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded
...,...,...,...
28293907,1206425,trogdorfamily7622,funded
28293908,1206425,danny6470,funded
28293909,1206425,don6118,funded
28293910,1206486,alan5175,funded


In [ ]:
#per prima cosa isolo le righe che hanno come valore dell'attributo status "funded", dopodichè, per ogni loan_id, 
# effettuo un conteggio (verifico la dimensione) del numero di righe in cui appare funded. 
#Infine isolo i loan_id che hanno finanziato almeno due volte (>=2)
dataframe_unito_funded = dataframe_unito[dataframe_unito['status'] == 'funded']
dataframe_unito_funded.head()

,loan_id,lenders,status
0,483693,muc888,funded
1,483693,sam4326,funded
2,483693,camaran3922,funded
3,483693,lachheb1865,funded
4,483693,rebecca3499,funded


In [ ]:
#utilizzo la funzione group by (sull'attributo lenders) sul dataframe con status funded e lenders divisi. Dopodichè utilizzo 
#la condizione >=2 per poter visualizzare i lenders che hanno finanziato almeno due volte
dataframe_unito_funded= dataframe_unito_funded.groupby('lenders')['lenders'].size()
dataframe_unito_funded.head()
dataframe_unito_funded[dataframe_unito_funded >= 2]

lenders
 000               39
 00000             38
 0002              70
 0101craign0101    71
 0132575            4
                   ..
zyrah8525           4
zyrorl              3
zzaman              9
zzanita             2
zzmcfate           54
Name: lenders, Length: 993782, dtype: int64

In [ ]:
#punto 4)
#For each country, compute how many loans have involved that country as borrowers.
#lavoriamo sul dataset 'loans'
#visualizzo il dataset loans utilzzando solamente le variabili 'loan_id' e 'country_name'
loans_num_lenders = loans[['loan_id','country_name']]
loans_num_lenders.groupby('country_name')['loan_id'].count()

country_name
Afghanistan        2337
Albania            3075
Armenia           13952
Azerbaijan        10172
Belize              218
                  ...  
Vietnam           21839
Virgin Islands        2
Yemen              4206
Zambia             1277
Zimbabwe           5513
Name: loan_id, Length: 96, dtype: int64

In [ ]:
#5)For each country, compute the overall amount of money borrowed.
#utilizzo il dataset 'loans', con la funzione groupby raggruppo per ogni paese e con la funzione 
#sum() eseguo una somma. 
#Isolo l'attributo 'loan_amount' per ogni paese ottenendo l'output richiesto 
calcolo_ammontare_totale = loans.groupby('country_name').sum()
calcolo_ammontare_totale.head()
calcolo_ammontare_totale_finale = calcolo_ammontare_totale[['loan_amount']]
calcolo_ammontare_totale_finale.head()

,loan_amount
country_name,
Afghanistan,1967950.0
Albania,4307350.0
Armenia,22950475.0
Azerbaijan,14784625.0
Belize,150175.0


In [ ]:
#6) Like the previous point, but expressed as a percentage of the overall amount lent.
#per prima calcolo il totale dell'ammontare dei prestiti, dopodichè calcolo per ogni paese
#(funzione groupby) l'ammontare di soldi prestati.
#Calcolo infine la percentuale 
totale = loans['loan_amount'].sum()
totale 

1181437300.0

In [ ]:
#calcolo la somma dell'ammontare del prestito per ogni paese, dopodichè lo divido per il totale e moltiplico per 100
conteggio_somma_per_paese = loans.groupby('country_name')['loan_amount'].sum().rename("conteggio")
percentuale = conteggio_somma_per_paese/totale*100
percentuale.head()

country_name
Afghanistan    0.166573
Albania        0.364586
Armenia        1.942589
Azerbaijan     1.251410
Belize         0.012711
Name: conteggio, dtype: float64

In [ ]:
#7)Like the three previous points, but split for each year (with respect to disburse time).
#per ogni paese e per ogni anno calcolo la percentuale di soldi 
#utilizzo i comandi precedentemente usati raggruppando però 'country_name' e 'disburse_time'. SI effettua quindi prima una somma
#per ogni riga raggruppata per i due attributi e si divide poi per il totale(calcolato nel punto precedente) e moltiplico 
#per 100
conteggio_anno_paese = loans.groupby(['country_name', 'disburse_time'])['loan_amount'].sum().rename("conteggio")
conteggio_anno_paese.head()
calcolo_percentuale = conteggio_anno_paese/totale*100
calcolo_percentuale

country_name  disburse_time                
Afghanistan   2007-02-02 00:12:07.000 +0000    0.000015
              2007-02-02 00:52:28.000 +0000    0.000015
              2007-02-02 03:56:02.000 +0000    0.000015
              2007-02-06 02:20:41.000 +0000    0.000015
              2007-02-06 03:52:07.000 +0000    0.000015
                                                 ...   
Zimbabwe      2017-12-20 08:00:00.000 +0000    0.000076
              2017-12-21 08:00:00.000 +0000    0.000157
              2017-12-27 08:00:00.000 +0000    0.000025
              2017-12-28 08:00:00.000 +0000    0.000135
              2017-12-29 08:00:00.000 +0000    0.000034
Name: conteggio, Length: 172583, dtype: float64

In [ ]:
#8)For each lender, compute the overall amount of money lent. For each loan that has more than one lender,
#you must assume that all lenders contributed the same amount.
#unisco il dataset loans con attributi 'loan_id','loan_amount','num_lenders_total' e il datset 
#loans_lenders per poter avere gli elementi richiesti 
loans_merge = pd.merge(loans[['loan_id','loan_amount','num_lenders_total']], loans_lenders[['lenders','loan_id']], on='loan_id')
loans_merge.head()

,loan_id,loan_amount,num_lenders_total,lenders
0,657307,125.0,3,spencer5657
1,657307,125.0,3,matthew8640
2,657307,125.0,3,larry71496105
3,657259,400.0,11,ltr
4,657259,400.0,11,andrew5306


In [ ]:
#creo una colonna chiamata "quota_singola" che indica la quota singola prestata
#da un lenders(istituto di credito)
#si calcola poi per ogni lenders la somma di quota singola versta, rinomino infine la colonna  
loans_merge['quota_singola'] = loans_merge['loan_amount'] / loans_merge['num_lenders_total']
#creo un dataframe con la quota totale versata da ogni istituto di credito 
prestito_totale = pd.DataFrame(loans_merge.groupby('lenders',as_index=False)['quota_singola'].sum())
#rinomino la colonna lenders in 'permanent_name' in modo tale da poter unire in seguito il dataset lenders con il dataset appena trovato 
prestito_totale.columns = ['permanent_name', 'total_lent']
prestito_totale.head()

,permanent_name,total_lent
0,000,1485.309656
1,00000,1249.947363
2,0002,2201.180463
3,00mike00,38.461538
4,0101craign0101,2424.088932


In [ ]:
#9)For each country, compute the difference between the overall amount of money lent and the overall amount 
#of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution 
#among the countries is the same as the one computed from the rows where the country is known.
#carico lenders isolando, per miglior chiarezza gli attributi utili 
lenders = pd.read_csv('/content/drive/My Drive/lenders.csv')
lenders = lenders[['permanent_name', 'country_code', 'occupation']]
lenders.head()

,permanent_name,country_code,occupation
0,qian3013,NaN,NaN
1,reena6733,NaN,NaN
2,mai5982,NaN,NaN
3,andrew86079135,NaN,NaN
4,nguyen6962,NaN,NaN


In [ ]:
#contiamo il numero di volte che appare il codice paese
lenders.groupby('country_code').size()

country_code
AD      15
AE    1043
AF     228
AG       8
AI       4
      ... 
YE     195
YT       2
ZA    1051
ZM      65
ZW      54
Length: 234, dtype: int64

In [ ]:
#osserviamo che sono presenti un gran numero di valori nulli, come da richiesta per risolvere il problema
#si ipotizza che questi ultimi seguino la stessa distribuzione dei valori a noi conosciuti. 
#utilizzando la funzione value_counts con normalize=true e sort=true si restituiscono le frequenze 
#relative del country_code ordinandole per frequenza   
countrycode_sort=lenders['country_code'].value_counts(normalize=True, sort=True)
countrycode_sort.head()

US    0.664330
CA    0.076325
GB    0.043097
AU    0.041664
DE    0.017975
Name: country_code, dtype: float64

In [ ]:
#una volta trovata frequenza con cui appare ogni coutry_code l'obiettivo è assegnare in modo 
#proporzionale i valori nulli alla distribuzione trovata 
#si procede quindi calcolando i valori nulli, dopodichè elimino i valori nulli (dropna), e tramite 
#funzione sample li rimpiazzo con i valori in modo proporzionale alla distribuzione
calcolo_valore_nullo = lenders['country_code'].isnull()
campione = lenders['country_code'].dropna().sample(calcolo_valore_nullo.sum(), replace=True).values
lenders.loc[calcolo_valore_nullo, 'country_code'] = campione
lenders.head() 

,permanent_name,country_code,occupation
0,qian3013,GB,NaN
1,reena6733,US,NaN
2,mai5982,US,NaN
3,andrew86079135,CA,NaN
4,nguyen6962,CA,NaN


In [ ]:
#per poter osservare se il comando ha effettivamente funzionato ricalcoliamo la frequenza del dataset 
#"rimpiazzato" osservando valori estremamente simili 
countrycode_sort=lenders['country_code'].value_counts(normalize=True, sort=True)
countrycode_sort.head()

US    0.664031
CA    0.076372
GB    0.043115
AU    0.041808
DE    0.017980
Name: country_code, dtype: float64

In [ ]:
#unisco il dataset trovato con il dataset comprendente il valore totale prestato per ogni istituto
#di credito 
merge_slot = pd.merge(lenders, prestito_totale, on='permanent_name' )
merge_slot.head()

,permanent_name,country_code,occupation,total_lent
0,naresh2074,US,NaN,58.820663
1,jinxian2217,US,NaN,45.161290
2,sreedevi9583,US,NaN,25.000000
3,nicole98153893,US,NaN,28.571429
4,prashankpradeep8456,US,NaN,25.000000


In [ ]:
#dal dataset sopra creato sommo, per ogni paese l'ammontare di denaro prestato dagli istituti di credito 
#calcolo la somma di denaro ricevuto in prestito dal dataset loans sommando "loan_amount" per ogni country_code 
#le somme vengono poi inserite all'interno di due dataframe, uniti e poi calcolata la differenza
somma_prestatori = pd.DataFrame(merge_slot.groupby('country_code').sum())
somma_prestito = pd.DataFrame(loans.groupby('country_code').sum()['loan_amount'])

,loan_amount
country_code,
AF,1967950.0
AL,4307350.0
AM,22950475.0
AZ,14784625.0
BA,477250.0


In [ ]:
frame_differenza = pd.merge(somma_prestatori,somma_prestito,on='country_code')
#si crea una nuova colonna per il mostrare il risulatato della differenza 
frame_differenza['differenza'] = round(frame_differenza['total_lent'] - frame_differenza['loan_amount'],0)
frame_differenza.head()

,total_lent,loan_amount,differenza
country_code,,,
AF,6468.703097,1967950.0,-1961481.0
AL,1544.767216,4307350.0,-4305805.0
AM,2122.643651,22950475.0,-22948352.0
AZ,616.356059,14784625.0,-14784009.0
BA,5716.367052,477250.0,-471534.0


In [ ]:
#10)Which country has the highest
#ratio between the difference computed at the previous point and the population?
#per poter eseguire la richiesta carico il dataset 'coutry_states' dove è presente il numero 
#della popolazione. Unisco il datset precedenemente trovato e calcolo il rapporto richiesto 
#infine individuo il valore più alto 

In [ ]:
#carico il dataset 
country_stats = pd.read_csv('/content/drive/My Drive/country_stats.csv')
country_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174 entries, 0 to 173
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   country_name                   174 non-null    object 
 1   country_code                   173 non-null    object 
 2   country_code3                  173 non-null    object 
 3   continent                      174 non-null    object 
 4   region                         174 non-null    object 
 5   population                     174 non-null    int64  
 6   population_below_poverty_line  152 non-null    float64
 7   hdi                            171 non-null    float64
 8   life_expectancy                168 non-null    float64
 9   expected_years_of_schooling    168 non-null    float64
 10  mean_years_of_schooling        168 non-null    float64
 11  gni                            168 non-null    float64
 12  kiva_country_name              174 non-null    obj

In [ ]:
#unisco il datset precedentemente trovato con country_stats prendendo in considerazione 
#solo la popolazione 
country_differenza = pd.merge(frame_differenza, country_stats[['population','country_code']], on='country_code')
country_differenza.head() 

,country_code,total_lent,loan_amount,differenza,population
0,AF,6468.703097,1967950.0,-1961481.0,35530081
1,AL,1544.767216,4307350.0,-4305805.0,2930187
2,AM,2122.643651,22950475.0,-22948352.0,2930450
3,AZ,616.356059,14784625.0,-14784009.0,9827589
4,BA,5716.367052,477250.0,-471534.0,3507017


In [ ]:
#calcolo del rapporto 
country_differenza['rapporto'] = country_differenza['differenza']/country_differenza['population']
country_differenza[['rapporto','country_code']].max()

rapporto        0.135725
country_code          ZW
dtype: object

In [ ]:
#11) Which country has the highest ratio between 
#the difference computed at point 9 and the population that is not below the poverty line?

In [ ]:
#unisco il datset creato al punto 9 con le variabili 'popualation' e 'population_below_poverty_line'
#del dataset country_states
#dopodichè creo una una colonna che comprende la percentuale di perone sopra la soglia della povertà 
#(100-'population_below_poverty_line'). 
#creo poi una seconda colonna con il numero di persone sopra la soglia di povertà

In [ ]:
dataset_over_poverty = pd.merge(frame_differenza, country_stats[['population','country_code','population_below_poverty_line']], on='country_code')
dataset_over_poverty.head()

,country_code,total_lent,loan_amount,differenza,population,population_below_poverty_line
0,AF,6468.703097,1967950.0,-1961481.0,35530081,35.8
1,AL,1544.767216,4307350.0,-4305805.0,2930187,14.3
2,AM,2122.643651,22950475.0,-22948352.0,2930450,32.0
3,AZ,616.356059,14784625.0,-14784009.0,9827589,4.9
4,BA,5716.367052,477250.0,-471534.0,3507017,17.2


In [ ]:
dataset_over_poverty['population_over_poverty_line'] = 100 - dataset_over_poverty['population_below_poverty_line']
dataset_over_poverty.head()

,country_code,total_lent,loan_amount,differenza,population,population_below_poverty_line,population_over_poverty_line
0,AF,6468.703097,1967950.0,-1961481.0,35530081,35.8,64.2
1,AL,1544.767216,4307350.0,-4305805.0,2930187,14.3,85.7
2,AM,2122.643651,22950475.0,-22948352.0,2930450,32.0,68.0
3,AZ,616.356059,14784625.0,-14784009.0,9827589,4.9,95.1
4,BA,5716.367052,477250.0,-471534.0,3507017,17.2,82.8


In [ ]:
dataset_over_poverty['numeber_pop_over_poverty_line'] = round(dataset_over_poverty['population']*dataset_over_poverty['population_over_poverty_line']/100,0)
dataset_over_poverty.head()

,country_code,total_lent,loan_amount,differenza,population,population_below_poverty_line,population_over_poverty_line,numeber_pop_over_poverty_line,ratio_pop
0,AF,6468.703097,1967950.0,-1961481.0,35530081,35.8,64.2,22810312.0,-0.085991
1,AL,1544.767216,4307350.0,-4305805.0,2930187,14.3,85.7,2511170.0,-1.714661
2,AM,2122.643651,22950475.0,-22948352.0,2930450,32.0,68.0,1992706.0,-11.516175
3,AZ,616.356059,14784625.0,-14784009.0,9827589,4.9,95.1,9346037.0,-1.581848
4,BA,5716.367052,477250.0,-471534.0,3507017,17.2,82.8,2903810.0,-0.162385


In [ ]:
dataset_over_poverty['ratio_pop'] = dataset_over_poverty['differenza']/dataset_over_poverty['numeber_pop_over_poverty_line']
dataset_over_poverty[['country_code','ratio_pop']].max() 

country_code          ZW
ratio_pop       0.149807
dtype: object

In [ ]:
#12)For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different 
#years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time
# December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 
#5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018. 

In [ ]:
#carico il dataset
loans_12 = loans[['loan_amount','disburse_time','planned_expiration_time']]
punto_12=pd.DataFrame(loans[['loan_amount','disburse_time','planned_expiration_time']])

In [ ]:
#elimino i valori nulli 
punto_12 = punto_12.dropna()
#converto in datetime
punto_12["disburse_time"]=pd.to_datetime(punto_12["disburse_time"])
punto_12["planned_expiration_time"]=pd.to_datetime(punto_12["planned_expiration_time"])
punto_12.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1044962 entries, 0 to 1419606
Data columns (total 3 columns):
 #   Column                   Non-Null Count    Dtype              
---  ------                   --------------    -----              
 0   loan_amount              1044962 non-null  float64            
 1   disburse_time            1044962 non-null  datetime64[ns, UTC]
 2   planned_expiration_time  1044962 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](2), float64(1)
memory usage: 31.9 MB


In [ ]:
#calcoliamo il range di anni per ogni riga del dataset rispetto disbursetime e planned_expiration_time
max_year=punto_12[['disburse_time','planned_expiration_time']].max().max().year
min_year=punto_12[['disburse_time','planned_expiration_time']].min().min().year

In [ ]:
#aggiungo una colonna relativa all'anno per ogni riga del dataset 
years=[str(year) for year in range(min_year,max_year+1)]

In [ ]:
#inserisco in ogni colonna relativa agli anni il numero 0
punto_12[years]=pd.DataFrame(np.zeros((len(punto_12),len(years))),columns=years)
punto_12 = punto_12.dropna()
punto_12.head()

,loan_amount,disburse_time,planned_expiration_time,2011,2012,2013,2014,2015,2016,2017,2018
0,125.0,2013-12-22 08:00:00+00:00,2014-02-14 03:30:06+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,400.0,2013-12-20 08:00:00+00:00,2014-03-26 22:25:07+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,400.0,2014-01-09 08:00:00+00:00,2014-02-15 21:10:05+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,625.0,2014-01-17 08:00:00+00:00,2014-02-21 03:10:02+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,425.0,2013-12-17 08:00:00+00:00,2014-02-13 06:10:02+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def days_in_year(row):
    
        date=row.disburse_time 
        end=row.planned_expiration_time
       
        while date<end:
            next_date=min(date+YearBegin(),end+Day())
            
            row[str(date.year)]=(next_date-date).days
           
            date=next_date
        
        return row

In [ ]:
#assegniamo il numero di giorni ad ogni anno 
punto_12=punto_12.apply(days_in_year,axis=1)

In [ ]:
#converto gli attributi da float ad int 
punto_12= punto_12[['loan_amount','2011','2012','2013','2014','2015','2016','2017','2018']].astype(int) 

In [ ]:
#calcolo per ogni anno l'ammontare di quota 
punto_12[years]=punto_12.apply(lambda row: row.loan_amount*row[years]/(row[years].sum()),axis=1).round(2)
punto_12.head()

,loan_amount,2011,2012,2013,2014,2015,2016,2017,2018
0,125,0.0,0.0,23.15,101.85,0.0,0.0,0.0,0.0
1,400,0.0,0.0,49.48,350.52,0.0,0.0,0.0,0.0
2,400,0.0,0.0,0.00,400.00,0.0,0.0,0.0,0.0
3,625,0.0,0.0,0.00,625.00,0.0,0.0,0.0,0.0
4,425,0.0,0.0,109.91,315.09,0.0,0.0,0.0,0.0


In [ ]:
#13) For each value of repayment_interval, add a new column to the lenders dataframe that contains
#the total amount of money corresponding to loans in such state

In [ ]:
#per poter risolvere il punto abbiamo bisogno di un dataset che contiene il nome del lenders, la quota singola prestata, il repayment 
#interval, loan_amount
#unisco il dataset loans con gli attributi repayment_interval e il dataset trovato al punto 8 con la quota singola per ogni lenders
dataset_repayment_interval = pd.merge(loans[['loan_id','repayment_interval']],loans_merge, on='loan_id')
dataset_repayment_interval.head()

,loan_id,repayment_interval,loan_amount,num_lenders_total,lenders,quota_singola
0,657307,irregular,125.0,3,spencer5657,41.666667
1,657307,irregular,125.0,3,matthew8640,41.666667
2,657307,irregular,125.0,3,larry71496105,41.666667
3,657259,monthly,400.0,11,ltr,36.363636
4,657259,monthly,400.0,11,andrew5306,36.363636


In [ ]:
#effettuo la somma raggruppata per lenders e per repayment_interval
dataset_repayment_interval = dataset_repayment_interval.groupby(['lenders','repayment_interval']).sum()
dataset_repayment_interval.head()
#trasformo in colonne le righe "bullet","irregular","monthly","weekly" 
dataset_repayment_interval.columns=['bullet','irregular','monthly','weekly']
dataset_repayment_interval=dataset_repayment_interval.reset_index()
dataset_repayment_interval.head()

,lenders,repayment_interval,bullet,irregular,monthly,weekly
0,000,bullet,2797082,6475.0,198,132.759846
1,000,irregular,3200797,12025.0,347,274.915733
2,000,monthly,16660673,85250.0,1984,1077.634077
3,00000,bullet,491565,3550.0,125,87.429825
4,00000,irregular,2738278,27075.0,841,520.199555


In [ ]:
#14)What is the occupation with the highest average amount of money lent
#(the average must be computed over all lenders with a given occupation)?

#per risolvere il punto utilizzo il dataset trovato al punto 8) composto da permanent_name (ex lenders) che indica il nome degli
#istituti di credito e l'ammontare totale di denaro prestato. Unisco quest'ultimo al dataset lenders prendendo in considerazione 
#solamente l'attributo occupation
occupation_dataset = pd.merge(prestito_totale, lenders[['permanent_name','occupation']], on='permanent_name')
occupation_dataset.head()

,permanent_name,total_lent,occupation
0,000,78.571429,NaN
1,0161130,25.000000,NaN
2,0169713,25.000000,NaN
3,0197462,25.000000,NaN
4,0206338,29.687500,NaN


In [ ]:
#calcolo del valore medio per total_lent
occupation_dataset['media'] = occupation_dataset.groupby('occupation').agg({'total_lent':['mean']})
occupation_dataset['media'].max()

nan

In [ ]:
#15) Cluster the loans according to the year-month of disburse time.
#creiamo due diverse colonne una per l'anno e una per il mese utilizzano la funzione str.split con 
#attributo di separazione '-' in modo tale da isolare anno e mese.
#sommo loan_amount per ogni mese e anno per raggruppare i prestiti come da richiesto 
loans['ammontare_annuo'] = loans['disburse_time'].str.split('-', expand= True)[0] 
loans['ammontare_mensile'] = loans['disburse_time'].astype(str).str.split('-', expand= True)[1]
loans.groupby(['ammontare_annuo','ammontare_mensile'])['loan_amount'].sum()

ammontare_annuo  ammontare_mensile
2005             04                     102850.0
2006             03                       1000.0
                 05                      42325.0
                 06                      33325.0
                 07                      41825.0
                                         ...    
2017             11                   14718375.0
                 12                    6031300.0
2018             01                     459250.0
                 02                     496475.0
                 03                      34300.0
Name: loan_amount, Length: 145, dtype: float64

In [ ]:
#16)moltiplico il GNI pro capite per il numero di popolazione in ogni country_name. 
#Non utilizzo il groupby o funzione aplly() perchè ogni nazione('country_name') appare una sola volta nel dataset  
country_stats['GNI'] = round(country_stats['gni']*country_stats['population'],0)
country_stats.head()

,country_name,country_code,country_code3,continent,region,population,population_below_poverty_line,hdi,life_expectancy,expected_years_of_schooling,mean_years_of_schooling,gni,kiva_country_name,GNI
0,India,IN,IND,Asia,Southern Asia,1339180127,21.9,0.623559,68.322,11.696590,6.298834,5663.474799,India,7.584413e+12
1,Nigeria,NG,NGA,Africa,Western Africa,190886311,70.0,0.527105,53.057,9.970482,6.000000,5442.901264,Nigeria,1.038975e+12
2,Mexico,MX,MEX,Americas,Central America,129163276,46.2,0.761683,76.972,13.299090,8.554985,16383.106680,Mexico,2.116096e+12
3,Pakistan,PK,PAK,Asia,Southern Asia,197015955,29.5,0.550354,66.365,8.106910,5.089460,5031.173074,Pakistan,9.912214e+11
4,Bangladesh,BD,BGD,Asia,Southern Asia,164669751,31.5,0.578824,71.985,10.178706,5.241577,3341.490722,Bangladesh,5.502424e+11
